# Run inference on time to merge model trained previously


## What we did previously

In the previous [notebook](./03_model_training.ipynb) we trained machine learning models to classify a PR's `time_to_merge` into one of the 10 bins (or "classes"). We then deployed the model with the highest f1-score as a service using the model saved in s3.

## In this step

In this notebook, we are going to fetch the model that we previously trained, saved and stored in s3. We will send a payload to this model and see how it performs on the test data.
# Time to Merge Prediction Inference Service

In the previous notebook, we explored some basic machine learning models for predicting time to merge of a PR.

In [1]:
import os
import sys
import gzip
import json
import boto3
import datetime
import requests
from dotenv import load_dotenv, find_dotenv
from io import BytesIO

import joblib

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

load_dotenv(find_dotenv(), override=True)

import warnings
warnings.filterwarnings('ignore') 

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [2]:
class CephCommunication:
    """
    Class to establish communication with a ceph s3 bucket.
    It connects with the bucket and provides methods to read and write data in the parquet format.
    """

    def __init__(
        self, s3_endpoint_url, aws_access_key_id, aws_secret_access_key, s3_bucket
    ):
        self.s3_endpoint_url = s3_endpoint_url
        self.aws_access_key_id = aws_access_key_id
        self.aws_secret_access_key = aws_secret_access_key
        self.s3_resource = boto3.resource(
            "s3",
            endpoint_url=self.s3_endpoint_url,
            aws_access_key_id=self.aws_access_key_id,
            aws_secret_access_key=self.aws_secret_access_key,
        )
        self.bucket = s3_bucket
        ## Todo: Add try catch

    def upload_to_ceph(self, dataframe, s3_path, filename):
        """
        This helper function takes as input the data frame to be uploaded, and the output filename.
        It then saves the data frame in the defined ceph bucket.
        """
        parquet_buffer = BytesIO()
        dataframe.to_parquet(parquet_buffer)
        s3_obj = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        status = s3_obj.put(Body=parquet_buffer.getvalue())
        return status

    def read_from_ceph(self, s3_path, filename):
        """
        Helper function to read from ceph and see if the saved data is correct.
        """
        buffer = BytesIO()
        s3_object = self.s3_resource.Object(self.bucket, f"{s3_path}/{filename}")
        s3_object.download_fileobj(buffer)
        df_temp = pd.read_parquet(buffer)
        return df_temp


def save_to_disk(dataframe, path, filename):
    """
    Helper function to save the dataframe
    as a parquet file to disk.
    """
    dataset_base_path = Path(path)
    dataset_base_path.mkdir(parents=True, exist_ok=True)
    dataframe.to_parquet(f"{path}/{filename}")
    return True

In [3]:
## CEPH Bucket variables
## Create a .env file on your local with the correct configs,

ORG = os.getenv("GITHUB_ORG")
REPO = os.getenv("GITHUB_REPO")

## S3 bucket credentials
s3_endpoint_url = os.getenv("S3_ENDPOINT_URL")
s3_access_key = os.getenv("AWS_ACCESS_KEY_ID")
s3_secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")
s3_bucket = os.getenv("S3_BUCKET")

s3_input_data_path = os.getenv("CEPH_BUCKET_PREFIX")

REMOTE = os.getenv("REMOTE")
RAW_DATA_PATH = os.path.join(
    s3_input_data_path, "srcopsmetrics/bot_knowledge", ORG, REPO, "PullRequest.json"
)

In [4]:
output = []
local_input_data_path = "../../../data/raw/GitHub/PullRequest.json.gz"
if REMOTE:
    print("getting dataset from ceph")
    s3 = boto3.resource(
        "s3",
        endpoint_url=s3_endpoint_url,
        aws_access_key_id=s3_access_key,
        aws_secret_access_key=s3_secret_key,
    )
    content = s3.Object(s3_bucket, RAW_DATA_PATH)
    file = content.get()["Body"].read().decode("utf-8")

    prs = json.loads(file)

    for pr in prs.splitlines():
        output.append(json.loads(pr))

else:
    print("getting dataset from local")
    with gzip.open(local_input_data_path, "r") as f:
        prs = json.loads(f.read().decode("utf-8"))


pr_df = pd.DataFrame(output)

getting dataset from ceph


In [5]:
# github pr dataset collected using thoth's mi-scheduler
pr_df.head()

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
0,Add kepler team and edge demo repo.,Part of: https://github.com/operate-first/apps...,S,HumairAK,1665151016,1.665156e+09,HumairAK,1.665156e+09,HumairAK,1,1,"{'HumairAK': 10, 'sesheta': 131}","{'1134679037': {'author': 'harshad16', 'words_...","[lgtm, size/S]",[92b9a81b7ddb75868937d9c1c4e70e39868edd2a],[github-config.yaml],1.665156e+09,1.665156e+09,155
1,Adding Kristi Nikolla to Op1st org,None,XS,quaid,1665079592,1.665124e+09,sesheta,1.665124e+09,sesheta,8,1,"{'schwesig': 2, 'sesheta': 65}","{'1133959522': {'author': 'schwesig', 'words_c...","[size/XS, lgtm, approved]","[73bceee2db2576d90ce74c59566d3499af13f2c1, 298...",[github-config.yaml],1.665124e+09,1.665124e+09,154
2,Update github-config.yaml,None,XS,saadullah01,1664236227,1.664808e+09,sesheta,1.664808e+09,sesheta,1,1,"{'sesheta': 182, 'goern': 1, 'HumairAK': 1}",{},"[size/XS, lgtm, approved, hacktoberfest, hackt...",[b2925ff04d09e73e855c5722440c98c56cba045e],[github-config.yaml],NaN,NaN,153
3,add djach7 and robotsail,Signed-off-by: Oleg <97077423+RobotSail@users....,XS,RobotSail,1664218891,1.664219e+09,HumairAK,1.664219e+09,HumairAK,1,1,{'sesheta': 203},{},"[size/XS, needs-ok-to-test]",[874fe7535feffb78e975b8657fe6d2cfdf4fadb0],[github-config.yaml],NaN,NaN,152
4,creating a utility-scripts repo,An action item of today's cluster ops sync. \r...,XS,Gregory-Pereira,1663080809,1.663099e+09,sesheta,1.663099e+09,sesheta,1,1,"{'Gregory-Pereira': 1, 'HumairAK': 2, 'sesheta...",{},"[size/XS, lgtm, approved]",[fa5d85540941b7fd92e94cc5f96bd4e677d75867],[github-config.yaml],NaN,NaN,151


In [6]:
interval = (pr_df["merged_at"] - pr_df["created_at"]).astype("float")
interval = interval.dropna()
interval

0        4702.0
1       44879.0
2      571464.0
3          17.0
4       18114.0
         ...   
138    120147.0
139    333546.0
140      1877.0
141      8112.0
142     92686.0
Length: 130, dtype: float64

In [7]:
n_buckets = 10

quantiles = interval.quantile(q=np.arange(0, 1 + 1e-100, 1 / n_buckets))
quantiles

0.0        17.0
0.1       207.2
0.2       506.8
0.3      1505.3
0.4      2848.0
0.5      7322.0
0.6     38129.6
0.7     77171.2
0.8    266684.2
0.9    507933.9
dtype: float64

In [8]:
quantiles / 3600

0.0      0.004722
0.1      0.057556
0.2      0.140778
0.3      0.418139
0.4      0.791111
0.5      2.033889
0.6     10.591556
0.7     21.436444
0.8     74.078944
0.9    141.092750
dtype: float64

In [9]:
time_intervals = quantiles / 3600

In [10]:
# remove PRs from train/test which are still open
pr_df = pr_df[pr_df["closed_at"].notna()]
pr_df = pr_df[pr_df["merged_at"].notna()]

In [11]:
pr_df["created_at"] = pr_df["created_at"].apply(
    lambda x: int(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["closed_at"] = pr_df["closed_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)
pr_df["merged_at"] = pr_df["merged_at"].apply(
    lambda x: float(datetime.datetime.timestamp(pd.to_datetime(x)))
)

In [12]:
TEST_DATA_PATH = os.path.join(s3_input_data_path, ORG, REPO, "test-data")

# read processed and split data created for train/test in the model training notebook
if REMOTE:
    cc = CephCommunication(s3_endpoint_url, s3_access_key, s3_secret_key, s3_bucket)
    X_test = cc.read_from_ceph(TEST_DATA_PATH, "X_test.parquet")
    y_test = cc.read_from_ceph(TEST_DATA_PATH, "y_test.parquet")

else:
    print(
        "The X_test.parquet and y_test.parquet files are not included in the github repo."
    )
    print(
        "Please set REMOTE=1 in the .env file and read this data from the S3 bucket instead."
    )

In [13]:
X_test

,size,created_at_day,created_at_month,created_at_weekday,created_at_hour,changed_files_number,body_size,commits_number,filetype_.yaml,filetype_.md,...,title_wordcount_u,title_wordcount_unnecessary,title_wordcount_updated,title_wordcount_user,title_wordcount_users,title_wordcount_vannten,title_wordcount_website,title_wordcount_websites,title_wordcount_work,title_wordcount_yaml
57,2.0,14.0,4.0,3.0,20.0,1.0,43.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42,0.0,26.0,5.0,3.0,14.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20,0.0,13.0,7.0,2.0,13.0,1.0,10.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
32,1.0,7.0,6.0,1.0,13.0,1.0,40.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,0.0,20.0,9.0,0.0,14.0,1.0,42.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
60,0.0,13.0,4.0,2.0,17.0,1.0,3.0,1.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0.0,18.0,3.0,4.0,15.0,1.0,32.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111,0.0,6.0,10.0,2.0,15.0,1.0,7.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0.0,8.0,2.0,1.0,18.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,13.0,6.0,0.0,16.0,1.0,7.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
y_test

,ttm_class
57,8
42,8
20,3
32,4
124,0
60,8
73,5
111,2
86,3
27,3


In [15]:
sample_payload = pr_df.reindex(X_test.index)

In [16]:
sample_payload

,title,body,size,created_by,created_at,closed_at,closed_by,merged_at,merged_by,commits_number,changed_files_number,interactions,reviews,labels,commits,changed_files,first_review_at,first_approve_at,id
57,Labels related to SIGs,Adds labels for existing SIGs from the new cha...,M,Gregory-Pereira,1,1.650299,sesheta,1.650299,sesheta,1,1,"{'quaid': 1, 'schwesig': 3, 'sesheta': 68}",{},"[size/M, lgtm, approved]",[267782ab90e228b0ebfe96c624674e3b447dba37],[labels.yaml],NaN,NaN,94
42,Add jpweiser to orgs.operate-first.members,None,XS,jpweiser,1,1.654012,HumairAK,1.654012,HumairAK,1,1,{'sesheta': 202},{},"[size/XS, needs-ok-to-test]",[6b713b1ead4c0ad7b425a318c8f6f1c17db1ce8a],[github-config.yaml],NaN,NaN,111
20,feat: Add robozome repo,Add https://github.com/operate-first/robozome ...,XS,tumido,1,1.657721,sesheta,1.657721,sesheta,1,1,"{'larsks': 1, 'sesheta': 68}","{'1037415916': {'author': 'schwesig', 'words_c...","[size/XS, lgtm, approved]",[f65762c67825694e8e6a6bdbf575bd3ddddfbe5e],[github-config.yaml],1.657721e+09,1.657721e+09,135
32,fix: Resolve errors in peribolos config,Peribolos is complaining that `mhild` GitHub a...,S,tumido,1,1.654612,sesheta,1.654612,sesheta,1,1,{'sesheta': 65},"{'998159147': {'author': 'schwesig', 'words_co...","[lgtm, size/S, approved]",[9693537d06ecaa2ac3fdba306b1ffe67675f3e0e],[github-config.yaml],1.654609e+09,1.654609e+09,123
124,feat: Remove Aaaaazzaa user which no longer ex...,"Breaks the automation runs, since the user no ...",XS,tumido,1,1.632148,sesheta,1.632148,sesheta,1,1,"{'HumairAK': 1, 'sesheta': 68}","{'758702888': {'author': '4n4nd', 'words_count...","[size/XS, lgtm, approved]",[5f09d84b1aa740d8a2d5c4ab63f1f55328774b28],[github-config.yaml],1.632148e+09,1.632148e+09,21
60,Adding SRE-U repo,Adding SRE-U repo,XS,marroblee,1,1.650377,HumairAK,1.650377,HumairAK,1,1,"{'sesheta': 274, 'marroblee': 27, 'HumairAK': 3}","{'942659707': {'author': 'schwesig', 'words_co...","[size/XS, lgtm, approved]",[fcdd368854e733f15b4786a40c3690c316054f41],[github-config.yaml],1.649961e+09,1.649961e+09,91
73,Add myself @stefwalter as a member of Operate ...,"This would allow me be assigned issues, create...",XS,stefwalter,1,1.647626,sesheta,1.647626,sesheta,1,1,"{'sesheta': 182, 'harshad16': 2}","{'914672104': {'author': 'harshad16', 'words_c...","[size/XS, lgtm, approved, ok-to-test]",[16684c35c6fbd048590fa6e6670dd7d35de27459],[github-config.yaml],1.647625e+09,1.647626e+09,76
111,Update repo description for support repo.,"Description is outdated, needs to be updated.",XS,HumairAK,1,1.633535,sesheta,1.633535,sesheta,1,1,{'sesheta': 65},"{'772916016': {'author': 'tumido', 'words_coun...","[size/XS, lgtm, approved]",[5cffe6672896267220bbdc0dc5848e7ac51bf6ca],[github-config.yaml],1.633535e+09,1.633535e+09,34
86,Add training github team.,SSIA,XS,HumairAK,1,1.644345,HumairAK,1.644345,HumairAK,1,1,{'sesheta': 183},"{'876471034': {'author': 'harshad16', 'words_c...","[size/XS, lgtm]",[4a9eaaba5c52c44744757349e47c79b0f9f00046],[github-config.yaml],1.644344e+09,1.644346e+09,62
27,Add two members of RH's SIG SRE team,So they can be added to sre/OWNERS,XS,mmazur,1,1.655138,sesheta,1.655138,sesheta,1,1,"{'schwesig': 2, 'sesheta': 68}","{'1004606638': {'author': 'schwesig', 'words_c...","[size/XS, lgtm, approved]",[95ff8610cf4e6ccb656020b433cd1ee094ac46f2],[github-config.yaml],1.655137e+09,1.655137e+09,128


In [17]:
sample_payload.dtypes

title                    object
body                     object
size                     object
created_by               object
created_at                int64
closed_at               float64
closed_by                object
merged_at               float64
merged_by                object
commits_number            int64
changed_files_number      int64
interactions             object
reviews                  object
labels                   object
commits                  object
changed_files            object
first_review_at         float64
first_approve_at        float64
id                       object
dtype: object

In [18]:
## read model
MODEL_KEY = os.path.join(s3_input_data_path, ORG, REPO, "ttm-model")
MODEL_FILENAME = "model.joblib"


s3_resource = boto3.resource(
    "s3",
    endpoint_url=s3_endpoint_url,
    aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key,
)

buffer = BytesIO()
s3_object = s3_resource.Object(s3_bucket, f"{MODEL_KEY}/{MODEL_FILENAME}")
s3_object.download_fileobj(buffer)
model = joblib.load(buffer)
model

Pipeline(steps=[('extract_features',
                 ColumnTransformer(transformers=[('pr_size',
                                                  OrdinalEncoder(categories=[['XS',
                                                                              'S',
                                                                              'M',
                                                                              'L',
                                                                              'XL',
                                                                              'XXL']]),
                                                  ['size']),
                                                 ('created_at_details',
                                                  DateTimeDetailsTransformer(),
                                                  ['created_at']),
                                                 ('n_changed_files',
                                                  NumChangedFilesTransformer(),
                                                  ['changed_files_number']),
                                                 ('body_size',
                                                  StringLenTransformer(),
                                                  ['body']),
                                                 ('n_commits', 'p...
                                                  TitleWordCountTransformer(words=['aaaaazzaa',
                                                                                   'abirami',
                                                                                   'access',
                                                                                   'acm',
                                                                                   'add',
                                                                                   'added',
                                                                                   'additional',
                                                                                   'admins',
                                                                                   'ai',
                                                                                   'ai4cloudops',
                                                                                   'andreatp',
                                                                                   'apicurio',
                                                                                   'approvers',
                                                                                   'areas',
                                                                                   'assign',
                                                                                   'avrajath',
                                                                                   'binary',
                                                                                   'blog',
                                                                                   'bootstrap',
                                                                                   'bot',
                                                                                   'branch',
                                                                                   'bruyland',
                                                                                   'changes',
                                                                                   'chore',
                                                                                   'christof',
                                                                                   'ci',
                                                                                   'clean',
                                                                                   'clearing',
           

In [19]:
# Test model on the dataset
preds = model.predict(sample_payload)
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         3
           3       1.00      0.25      0.40         4
           4       0.25      0.33      0.29         3
           5       0.11      0.50      0.18         2
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         7
           9       0.00      0.00      0.00         2

    accuracy                           0.12        26
   macro avg       0.14      0.11      0.09        26
weighted avg       0.19      0.12      0.11        26



# Conclusion

This notebook fetches the saved model from s3 and sends a payload to see how the model is performing on this new data. Additionally, we see that the evaluation scores in the classification report match the ones we saw in the training notebook. So, great, looks like our model are working as expected, and are ready to predict some times to merge for GitHub PRs! 